In [84]:
import os
import sys
import importlib
import json
import numpy as np
import pandas as pd
import scipy.ndimage as snd
import skimage
import uuid

if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir("../..")
directory_path = os.path.abspath(os.path.join("src"))
if directory_path not in sys.path:
    sys.path.append(directory_path)

import EyeTraumaAnalysis

print(directory_path)
importlib.reload(EyeTraumaAnalysis);
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl
import cv2

/Users/ryerrabelli/Library/CloudStorage/GoogleDrive-ryerrabelli@gmail.com/My Drive/Computer Backups/Rahul Yerrabelli drive/PythonProjects/EyeTraumaAnalysis/src


In [85]:
kmeans_labels = pd.read_excel("data/01_raw/Ergonautus/Ergonautus_Clusters_Correct_Values.xlsx", dtype={
    "Correct 1":"Int64", # "Int64" is from pandas, unlike int64 and allows null
    "Correct 2":"Int64",
    "Correct 3":"Int64",
    "Borderline":"Int64",
    "Notes":str,
    "Filename":str,
}, na_filter=False) # False na_filters make empty value for str column be "" instead of NaN

In [144]:
all_metrics = []
all_kmeans_masks = {}
for ind, filename in enumerate(kmeans_labels["Filename"]):
    img_bgr = skimage.io.imread(os.path.join("data/01_raw/",filename))
    centers, ranges, res_bgr, kmeans_masks = EyeTraumaAnalysis.kmeans.create_kmeans(img_bgr)
    metrics = EyeTraumaAnalysis.kmeans.get_kmeans_metrics(centers, ranges, kmeans_masks)
    all_metrics.append(metrics)
    all_kmeans_masks[filename] = kmeans_masks

all_metrics = pd.concat(all_metrics, keys=kmeans_labels["Filename"])


In [292]:
all_metrics.loc[:, ("Labels","Value","","")] = "False"
all_metrics.loc[:, ("Labels","Correct","","")] = False
all_metrics.loc[:, ("Labels","Borderline","","")] = False
for ind, row in kmeans_labels.iterrows():
    filename = row["Filename"]

    correct_indices =row[["Correct 1","Correct 2","Correct 3"]] - 1   # subtract by 1 to go from 1-10 to 0-9
    correct_indices = correct_indices[~pd.isnull(correct_indices)].to_numpy().astype(int)
    file_maskrow_correct_indices = [(filename,correct_index) for correct_index in correct_indices]
    all_metrics.loc[file_maskrow_correct_indices, ("Labels","Correct","","")]=True
    all_metrics.loc[file_maskrow_correct_indices, ("Labels","Value","","")]="True"

    borderline_indices = row[["Borderline"]] - 1
    borderline_indices = borderline_indices[~pd.isnull(borderline_indices)].to_numpy().astype(int)
    file_maskrow_borderline_indices = [(filename,borderline_index) for borderline_index in borderline_indices]
    all_metrics.loc[file_maskrow_borderline_indices, ("Labels","Borderline","","")]=True
    all_metrics.loc[file_maskrow_borderline_indices, ("Labels","Value","","")]="Maybe"

# Reorder
all_metrics = all_metrics[["Labels","Ranks","Values"]]
# Label the second index
all_metrics.index.names = [all_metrics.index.names[0], "Mask"]


In [151]:
all_metrics_agg = all_metrics.groupby([("Labels","Value")]).agg(["median"])[["Ranks","Values"]]

/Users/ryerrabelli/opt/anaconda3/envs/EyeTraumaAnalysis/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3135: PerformanceWarning: indexing past lexsort depth may impact performance.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/ryerrabelli/opt/anaconda3/envs/EyeTraumaAnalysis/lib/python3.10/site-packages/pandas/core/groupby/generic.py:979: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = op.agg()


# Plot

In [293]:
import plotly.express as px
#fig = px.box(all_metrics, x=("Labels","Value","",""), y=("Values","Color","Center","V"))
fig = px.box(y=all_metrics[("Values","Color","Center","V")].values,
             x=all_metrics[("Labels","Value")].values, points="all", notched=True, boxmode="group")
fig.show()

/Users/ryerrabelli/opt/anaconda3/envs/EyeTraumaAnalysis/lib/python3.10/site-packages/IPython/core/interactiveshell.py:2881: PerformanceWarning:

indexing past lexsort depth may impact performance.



# Create flat version of df

In [301]:
var_labels = {
    "Labels-Value": "Conjunctiva cluster",
    "Values-Color-Center-H": "Middle H",
    "Values-Color-Center-S": "Middle S",
    "Values-Color-Center-V": "Middle V",
}
plotly_template = "plotly_dark"

In [294]:
all_metrics_flat = all_metrics.copy()
all_metrics_flat.columns = ["-".join(multi_col).rstrip("-") for multi_col in all_metrics.columns]
all_metrics_flat = all_metrics_flat.reset_index()
all_metrics_flat.columns

Index(['Filename', 'Mask', 'Labels-Value', 'Labels-Correct',
       'Labels-Borderline', 'Ranks-Color-Center-H', 'Ranks-Color-Center-S',
       'Ranks-Color-Center-V', 'Ranks-Color-Range-H', 'Ranks-Color-Range-S',
       'Ranks-Color-Range-V', 'Ranks-Location-Mean-x', 'Ranks-Location-Mean-y',
       'Ranks-Location-SD-x', 'Ranks-Location-SD-y', 'Ranks-Area',
       'Values-Color-Center-H', 'Values-Color-Center-S',
       'Values-Color-Center-V', 'Values-Color-Range-H', 'Values-Color-Range-S',
       'Values-Color-Range-V', 'Values-Location-Mean-x',
       'Values-Location-Mean-y', 'Values-Location-SD-x',
       'Values-Location-SD-y', 'Values-Area'],
      dtype='object')

In [302]:
import plotly.express as px
#fig = px.box(all_metrics, x=("Labels","Value","",""), y=("Values","Color","Center","V"))
fig = px.box(all_metrics_flat, ("Labels-Value"), "Values-Color-Center-V", points="all",
             boxmode="group", notched=True, width=500, height=300, template=plotly_template, labels=var_labels)
fig.show()

In [303]:
import plotly.express as px
fig = px.scatter_matrix(all_metrics_flat,
                        dimensions=["Values-Color-Center-H", "Values-Color-Center-S", "Values-Color-Center-V",],
                        color="Labels-Value", template=plotly_template, labels=var_labels)
fig.update_traces(marker=dict(size=2))
fig.show()

In [306]:
import plotly.express as px
fig = px.scatter_3d(all_metrics_flat, x="Values-Color-Center-H", y="Values-Color-Center-S",
                    z="Values-Color-Center-V", color="Labels-Value", template=plotly_template, labels=var_labels)
fig.update_traces(marker=dict(size=2, opacity=0.5),
                  selector=dict(mode='markers'))
fig.show()

In [307]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter_matrix(all_metrics_flat,
                        dimensions=["Values-Location-Mean-x", "Values-Location-Mean-y",
                                    "Values-Location-SD-x", "Values-Location-SD-y"],
                        color="Labels-Value", template=plotly_template, labels=var_labels)
fig.update_traces(marker=dict(size=2))
fig.show()

In [268]:
import plotly.express as px
df = px.data.wind()
all_metrics_flat_temp = all_metrics_flat.copy()
all_metrics_flat_temp["Ranks-Color-Center-H"] = all_metrics_flat_temp["Ranks-Color-Center-H"] * 360/10
all_metrics_flat_temp["Values-Color-Center-H"] = all_metrics_flat_temp["Values-Color-Center-H"] * 360/256
fig = px.bar_polar(all_metrics_flat_temp, theta="Ranks-Color-Center-H", r="Ranks-Color-Center-S", color="Labels-Value", template="plotly_dark")
fig.show()

In [124]:
all_metrics.agg(["median","max"])

Values                                                         \
        Color                                     Location             
       Center                Range                    Mean             
            H      S      V      H      S      V         x         y   
median  111.0   93.5  111.5   72.0  118.0   96.0  0.490960  0.396355   
max     169.0  248.0  250.0  244.0  242.0  238.0  0.777552  0.834494   

                            ... Ranks                                         \
                            ... Color             Location                     
              SD            ... Range                 Mean          SD         
               x         y  ...     H     S     V        x     y     x     y   
median  0.169496  0.155001  ...   5.5   5.5   5.5      5.5   5.5   5.5   5.5   
max     0.378996  0.405495  ...  10.0  10.0  10.0     10.0  10.0  10.0  10.0   

              Labels             
        Area Correct Borderline  
                                 
                                 
median   5.5     0.0        0.0  
max     10.0    True       True  

[2 rows x 24 columns]

In [88]:
all_metrics

Values                                                          \
             Color                            Location                       
            Center           Range                Mean                  SD   
                 H    S    V     H    S    V         x         y         x   
Filename                                                                     
14000.png 0    132   62   48   188  175  172  0.515040  0.300133  0.104746   
          1    118  107   62    78  112  170  0.525279  0.371513  0.178863   
          2    123   56  109    75  168  111  0.555311  0.398597  0.176230   
          3    112  108  127    98  113   92  0.637666  0.418068  0.282251   
          4    133   34  145    73  180   87  0.467929  0.403017  0.139079   
...            ...  ...  ...   ...  ...  ...       ...       ...       ...   
14439.png 5    109   91  135    57  103   57  0.456736  0.453986  0.346920   
          6    114   34  147    78  159   58  0.462821  0.380127  0.202566   
          7    108   78  171    57  116   16  0.557086  0.612710  0.338504   
          8      0    1  189   162  162   73  0.404894  0.638407  0.081327   
          9    117   16  195    92  160    0  0.421598  0.597407  0.154638   

                       ...  Ranks                                          \
                       ...  Color               Location             Area   
                       ... Center Range             Mean      SD            
                    y  ...      V     H   S   V        x   y   x   y        
Filename               ...                                                  
14000.png 0  0.098856  ...      1     5   2  10       10   1   9   9    9   
          1  0.157227  ...      2     3   4   8        9   9  10   7   10   
          2  0.104414  ...      3    10   8   7        8   2   1   1    5   
          3  0.197722  ...      4     7   6   6        7   3   5   3    3   
          4  0.112933  ...      5     2   3   9        5   5   7   5    7   
...               ...  ...    ...   ...  ..  ..      ...  ..  ..  ..  ...   
14439.png 5  0.280295  ...      6     4   4   3        2   4  10  10    7   
          6  0.189435  ...      7     7   7   4        4   6   7   5    4   
          7  0.341271  ...      8    10  10   5        5  10   5   7   10   
          8  0.135791  ...      9     1   1   1        3   8   8   6    6   
          9  0.160964  ...     10     9   9   2        8   9   6   8    8   

            Labels  
                    
                    
                    
Filename            
14000.png 0  False  
          1  False  
          2  False  
          3  False  
          4  False  
...            ...  
14439.png 5  False  
          6  False  
          7  False  
          8  False  
          9  False  

[1450 rows x 23 columns]

# Test model

In [76]:
segmentations_trues = {}
clusters_trues = {}

for ind, row in kmeans_labels.iterrows():
    correct_indices =row[["Correct 1","Correct 2","Correct 3"]] - 1   # subtract by 1 to go from 1-10 to 0-9
    correct_indices = correct_indices[~pd.isnull(correct_indices)].to_numpy().astype(int)
    filename = row["Filename"]
    kmeans_masks = all_kmeans_masks[filename]
    # get combined masks of the clusters chosen. The .any() applies an OR so only a pixel needs to be in only one cluster
    # to be included in the combined mask
    segmentations_trues[filename] =  np.any(kmeans_masks[correct_indices],axis=0)
    clusters_trues[filename] =  correct_indices

In [147]:
segmentations_preds = {}
clusters_preds = {}

for ind, filename in enumerate(kmeans_labels["Filename"]):

    metrics = all_metrics.loc[filename]
    kmeans_masks = all_kmeans_masks[filename]
    chosen = EyeTraumaAnalysis.kmeans.choose_kmeans_cluster(metrics)

    # get combined masks of the clusters chosen. The .any() applies an OR so only a pixel needs to be in only one cluster
    # to be included in the combined mask
    segmentations_preds[filename] =  np.any(kmeans_masks[chosen.index],axis=0)
    clusters_preds[filename] =  np.array(chosen.index)

In [15]:
all_metrics

Values                                                          \
               Color                            Location                       
              Center           Range                Mean                  SD   
                   H    S    V     H    S    V         x         y         x   
File Number                                                                    
14000       0    145   46   40    52  153  148  0.472951  0.261635  0.067235   
            1    121   88   51   166  149  150  0.533301  0.374749  0.120643   
            2    117  122   57    56   62  148  0.511818  0.320543  0.168164   
            3    127   51   95    52  152  104  0.556200  0.356060  0.157804   
            4    114  109  104    57   88   89  0.571606  0.414762  0.285832   
...              ...  ...  ...   ...  ...  ...       ...       ...       ...   
14439       5    115   32  135    75  156   64  0.466713  0.369801  0.187856   
            6    108   84  164    54   99   21  0.572155  0.556399  0.360002   
            7    110   56  177    54  134   12  0.444744  0.678352  0.230992   
            8      0    1  189   159  159   70  0.404894  0.638407  0.081327   
            9    118   12  196    89  157    0  0.413804  0.593741  0.149003   

                         ...  Ranks                                         \
                         ...  Color                   Location               
                         ... Center     Range             Mean      SD       
                      y  ...      S   V     H   S   V        x   y   x   y   
File Number              ...                                                 
14000       0  0.055193  ...     10   1     1   3  10       10   1   1   1   
            1  0.128594  ...      8   2     4   7   9        9   3  10   4   
            2  0.180089  ...      1   3     8   5   8        1   4   2  10   
            3  0.101643  ...      4   4     3   9   7        8   2   4   8   
            4  0.160718  ...      6   5     5   2   6        3   5   8   6   
...                 ...  ...    ...  ..   ...  ..  ..      ...  ..  ..  ..   
14439       5  0.169744  ...      2   6     4   4   9        2   4   3   4   
            6  0.342594  ...      7   7     6   6   4        6   7   6   6   
            7  0.280577  ...      1   8    10  10   1        4  10   8   5   
            8  0.135791  ...      5   9     1   1   3        3   9   5   8   
            9  0.143836  ...      3  10     9   9   2        7   8   7   7   

                    
              Area  
                    
                    
File Number         
14000       0   10  
            1    1  
            2    6  
            3    8  
            4    3  
...            ...  
14439       5   10  
            6    4  
            7    8  
            8    5  
            9    7  

[1450 rows x 22 columns]

In [145]:
def calculate_prediction_statistics_areas(segmentations_trues, segmentations_preds):
    prediction_statistics = {}
    for filename in segmentations_trues.keys():
        mask_true = segmentations_trues[filename]
        mask_pred = segmentations_preds[filename]

        total_area = np.prod(mask_true.shape)

        intersection_area = true_positive_area = np.count_nonzero(mask_true & mask_pred) / total_area
        false_positive_area = np.count_nonzero(mask_true & ~mask_pred) / total_area
        false_negative_area = np.count_nonzero(~mask_true & mask_pred) / total_area
        true_negative_area = np.count_nonzero(~mask_true & ~mask_pred) / total_area
        union_area = true_positive_area + false_positive_area + false_negative_area

        # https://towardsdatascience.com/how-accurate-is-image-segmentation-dd448f896388
        # Jaccard's index: Intersection over union
        jaccard_area = intersection_area / union_area
        # dice index: Jaccard's index but double counting intersection
        dice_area = 2*intersection_area / (union_area + intersection_area)

        prediction_statistics[filename] = {
            "jaccard":jaccard_area,
            "dice": dice_area,
            "true_positive": true_positive_area,
            "false_positive": false_positive_area,
            "false_negative": false_negative_area,
            "true_negative": true_negative_area,
            "union": union_area,
        }

In [149]:
def calculate_prediction_statistics_clusters(clusters_true, clusters_pred, total_clusters=10):
    prediction_statistics = {}
    for filename in segmentations_trues.keys():
        clusters_true = clusters_trues[filename]
        clusters_pred = clusters_preds[filename]

        #intersection_clusters = true_positive_clusters = len([cluster for cluster in clusters_true if cluster in clusters_pred])
        intersection_clusters \
            = true_positive_clusters = len([cluster for cluster in range(total_clusters)
                                            if cluster in clusters_true and cluster in clusters_pred])
        false_positive_clusters = len([cluster for cluster in range(total_clusters)
                                       if cluster not in clusters_true and cluster in clusters_pred])
        false_negative_clusters = len([cluster for cluster in range(total_clusters)
                                       if cluster in clusters_true and cluster not in clusters_pred])
        true_negative_clusters  = len([cluster for cluster in range(total_clusters)
                                       if cluster not in clusters_true and cluster not in clusters_pred])
        union_clusters = true_positive_clusters + false_positive_clusters + false_negative_clusters

        # https://towardsdatascience.com/how-accurate-is-image-segmentation-dd448f896388
        # Jaccard's index: Intersection over union
        jaccard_clusters = intersection_clusters , union_clusters
        # dice index: Jaccard's index but double counting intersection
        dice_clusters = 2*intersection_clusters , (union_clusters + intersection_clusters)
        distribution_clusters = (true_positive_clusters, false_positive_clusters, false_negative_clusters, true_negative_clusters)
        prediction_statistics[filename] = jaccard_clusters

        prediction_statistics[filename] = {
            "jaccard":jaccard_clusters,
            "dice": dice_clusters,
            "true_positive": true_positive_clusters,
            "false_positive": false_positive_clusters,
            "false_negative": false_negative_clusters,
            "true_negative": true_negative_clusters,
            "union": union_clusters,
        }

    return prediction_statistics

In [151]:
prediction_statistic_area = calculate_prediction_statistics_areas(segmentations_trues, segmentations_preds)
prediction_statistic_clusters = calculate_prediction_statistics_clusters(clusters_trues, clusters_preds)
[prediction_statistic["jaccard"] for prediction_statistic in prediction_statistic_clusters.values()]

[(1, 3),
 (1, 3),
 (2, 3),
 (1, 3),
 (0, 4),
 (1, 3),
 (0, 4),
 (1, 3),
 (2, 2),
 (1, 3),
 (2, 2),
 (1, 3),
 (1, 3),
 (0, 4),
 (1, 3),
 (0, 5),
 (0, 3),
 (1, 2),
 (1, 2),
 (0, 4),
 (1, 3),
 (1, 3),
 (1, 3),
 (1, 2),
 (2, 2),
 (1, 3),
 (0, 4),
 (1, 3),
 (1, 3),
 (1, 3),
 (1, 2),
 (0, 4),
 (2, 2),
 (1, 2),
 (0, 3),
 (1, 2),
 (0, 3),
 (2, 2),
 (0, 3),
 (1, 2),
 (0, 3),
 (1, 3),
 (0, 3),
 (1, 2),
 (0, 3),
 (1, 2),
 (1, 2),
 (1, 3),
 (1, 2),
 (1, 2),
 (1, 2),
 (0, 3),
 (0, 3),
 (1, 3),
 (2, 2),
 (1, 2),
 (1, 2),
 (1, 3),
 (0, 3),
 (0, 3),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (0, 3),
 (1, 2),
 (1, 2),
 (0, 3),
 (1, 2),
 (2, 2),
 (1, 3),
 (2, 2),
 (1, 3),
 (1, 3),
 (1, 3),
 (1, 2),
 (1, 2),
 (1, 3),
 (1, 2),
 (1, 2),
 (1, 3),
 (1, 2),
 (1, 2),
 (1, 3),
 (1, 3),
 (0, 4),
 (0, 3),
 (0, 4),
 (1, 2),
 (1, 3),
 (1, 2),
 (1, 3),
 (0, 4),
 (1, 2),
 (0, 3),
 (2, 2),
 (0, 4),
 (1, 3),
 (1, 3),
 (1, 2),
 (0, 3),
 (0, 3),
 (1, 2),
 (1, 3),
 (0, 3),
 (1, 3),
 (1, 2),
 (0, 4),
 (1, 3),
 (2, 3),
 (1, 3),
 

In [11]:
all_metrics = pd.concat(all_metrics, keys=kmeans_labels["Filename"])

Values                                                          \
               Color                            Location                       
              Center           Range                Mean                  SD   
                   H    S    V     H    S    V         x         y         x   
File Number                                                                    
14000       0    145   46   40    52  153  148  0.472951  0.261635  0.067235   
            1    121   88   51   166  149  150  0.533301  0.374749  0.120643   
            2    117  122   57    56   62  148  0.511818  0.320543  0.168164   
            3    127   51   95    52  152  104  0.556200  0.356060  0.157804   
            4    114  109  104    57   88   89  0.571606  0.414762  0.285832   
...              ...  ...  ...   ...  ...  ...       ...       ...       ...   
14439       5    115   32  135    75  156   64  0.466713  0.369801  0.187856   
            6    108   84  164    54   99   21  0.572155  0.556399  0.360002   
            7    110   56  177    54  134   12  0.444744  0.678352  0.230992   
            8      0    1  189   159  159   70  0.404894  0.638407  0.081327   
            9    118   12  196    89  157    0  0.413804  0.593741  0.149003   

                         ...  Ranks                                         \
                         ...  Color                   Location               
                         ... Center     Range             Mean      SD       
                      y  ...      S   V     H   S   V        x   y   x   y   
File Number              ...                                                 
14000       0  0.055193  ...     10   1     1   3  10       10   1   1   1   
            1  0.128594  ...      8   2     4   7   9        9   3  10   4   
            2  0.180089  ...      1   3     8   5   8        1   4   2  10   
            3  0.101643  ...      4   4     3   9   7        8   2   4   8   
            4  0.160718  ...      6   5     5   2   6        3   5   8   6   
...                 ...  ...    ...  ..   ...  ..  ..      ...  ..  ..  ..   
14439       5  0.169744  ...      2   6     4   4   9        2   4   3   4   
            6  0.342594  ...      7   7     6   6   4        6   7   6   6   
            7  0.280577  ...      1   8    10  10   1        4  10   8   5   
            8  0.135791  ...      5   9     1   1   3        3   9   5   8   
            9  0.143836  ...      3  10     9   9   2        7   8   7   7   

                    
              Area  
                    
                    
File Number         
14000       0   10  
            1    1  
            2    6  
            3    8  
            4    3  
...            ...  
14439       5   10  
            6    4  
            7    8  
            8    5  
            9    7  

[1450 rows x 22 columns]

In [ ]:
centers, ranges, res_bgr, kmeans_masks = EyeTraumaAnalysis.kmeans.create_kmeans(img_bgr)
metrics = EyeTraumaAnalysis.kmeans.get_kmeans_metrics(centers, ranges, kmeans_masks)
chosen = EyeTraumaAnalysis.kmeans.choose_kmeans_cluster(metrics)
# get combined masks of the clusters chosen. The .any() applies an OR so only a pixel needs to be in only one cluster
# to be included in the combined mask
kmeans_masks_chosen = np.any(kmeans_masks[chosen.index],axis=0)

In [2]:
os.getcwd()

'/Users/ryerrabelli/Library/CloudStorage/GoogleDrive-ryerrabelli@gmail.com/My Drive/Computer Backups/Rahul Yerrabelli drive/PythonProjects/EyeTraumaAnalysis'